# Real Time Pipeline

Let us try to construct the real time pipe line using the constructed cascaded ML model

# Preposition Techniques

In [1]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pickle


def ff_transform(time_signal, frequency):

    '''- INPUTS -
    time_signal : the time domain matrix that to be converted into the fourier domain (it shoulf be in a 1D array of any size)
    frequency : the sampling frequency at which the eeg signals are captured

    OUTPUTS
    frequencies : 1D array containinf the frequencies of the output
    frequency_domain : the corresponding magnitutes are given as an array with respective to the frequencies. The output is given as a complex number
    '''
    # Perform Fourier transformation
    frequencies = np.fft.fftfreq(len(time_signal),d = 1/frequency)
    frequency_domain = np.fft.fft(time_signal)
    return frequencies, frequency_domain

def extract_harmonics(frequencies, frequency_domain, target_frequency, num_harmonics):
    ''' 
    -INPUTS
    frequencies, frequency_domain : same outputs as the ff_transform function
    target_frequency : the expected frequency, eg: when the target frequency is 5 the magnitudes of the closed components are intergrated
    num_harmonics : How many harmonics requered
    
    -OUTPUT
    returns a vector with magnitudes of the frequencies and harmonics.
    eg : if target_frequency = 5 & num_harmonics = 4
    the function outputs the magnitudes corresponding to [5,10,15,20]
    '''
    target_indices = []
    for i in range(num_harmonics):
        target_index = np.abs(frequencies-(i+1)*target_frequency).argmin()
        target_indices.append(target_index)
        #print(target_index)
    
    #feature_vector = np.zeros(num_harmonics)
    feature_vector = []
    for j in target_indices:
        feature = (frequency_domain[j] + frequency_domain[j+1] + frequency_domain[j-1] + frequency_domain[j-2]+frequency_domain[j+2])/5
        feature_vector.append(feature)
        #feature_vector[i] = feature

    #print(target_index)
    return np.array(feature_vector)

scaler = MinMaxScaler()
def all_features_per_channel(frequencies , frequency_domain, target_frequency_array, num_harmonics):
    ''' -INPUTS-
    frequencies, frequency_domain : same as the output of the ff_trainsform
    target_frequency_array : [6.5, 7,5, 8.5, 12]
    num_harmonics : 2

    - OUTPUTS
    1D array containing the features corresponds to [5,10,15,6,12,18,7,14,21,8,16,24] 
    '''
    all_features_of_channel = []
    for target_frequency in target_frequency_array:
        feature_vector = abs(extract_harmonics(frequencies,frequency_domain,target_frequency,num_harmonics))
        all_features_of_channel.append(feature_vector)
    all_features_of_channel = np.array(all_features_of_channel).flatten()
    data_2d = [[x] for x in all_features_of_channel]
    scaled_data = scaler.fit_transform(data_2d)
    all_features_of_channel = np.array([x[0] for x in scaled_data])
    all_features_of_channel_unscaled = np.array([x[0] for x in data_2d])
    return all_features_of_channel, all_features_of_channel_unscaled

def features(channel_signal,sampling_frequency,target_array, num_harmonics):
    """
    assuming that the channel_signal is in the format of (256*3, 16)
    [channel01 features, channel02 features, channel03 features, channel04 features...]
    """
    
    feeding_data = []
    feeding_data_unscaled = []
    for channel in range(channel_signal.shape[1]):
        frequencies, frequency_domain = ff_transform(channel_signal[:,channel],sampling_frequency)
        channel_features, channel_features_unscaled = all_features_per_channel(frequencies, frequency_domain, target_array, num_harmonics)
        feeding_data.append(channel_features)
        feeding_data_unscaled.append(channel_features_unscaled)
    
    feeding_array = np.concatenate(feeding_data, axis = 0).flatten()
    return feeding_array 

# Import the saved models

In [2]:
model_path1 = f"F:\Engineering\Semester 8\EE406 - Undergraduate Project 2\Saved Models\modeltrial1.pkl"
model_path2 = f"F:\Engineering\Semester 8\EE406 - Undergraduate Project 2\Saved Models\modeltrial2.pkl"
model_path3 = f"F:\Engineering\Semester 8\EE406 - Undergraduate Project 2\Saved Models\modeltrial3.pkl"
model_path4 = f"F:\Engineering\Semester 8\EE406 - Undergraduate Project 2\Saved Models\modeltrial4.pkl"

with open(model_path1, 'rb') as file:
    model1 = pickle.load(file)

with open(model_path2, 'rb') as file:
    model2 = pickle.load(file)

with open(model_path3,'rb') as file:
    model3 = pickle.load(file)

with open(model_path4,'rb') as file:
    model4 = pickle.load(file)

In [3]:
feature_array = np.random.rand(192).reshape(-1,1).T
result = model1.predict_proba(feature_array).flatten()
print(result[0])

0.4385590728288219


# Robot Arm Operations

In [31]:
#Don't code and change this code take as an example 
# 2017-05-26 by scharett, 
# very basic example of AL5 + Python
#First code to move robot arm into a selected direction
# obtain required libraries
import serial
import time

# create and open a serial port
sp = serial.Serial("COM3", 9600)

#Call joints using a loop setup initial position

Initial_position = 1500
def initial_position(initial_position):
    for i in range(5):
        command = f"#{i} P{Initial_position}\r"
        sp.write(command.encode())

#Rotate a servo motor in an axis into selected 
def rotate_servo(angle):
    return int(1500 + (angle)*(1000/90)) #500 is the scale of 90 degrees 

############################################################# edits
def servo_position(servo,position):
    command = f"#{servo} P{position}\r"
    sp.write(command.encode())

def smooth_move(servo,start_position,target_position,time_delay=0.05,steps=60):
    
    step_size = int((target_position-start_position)/steps)
    for step in range(steps):
        position = (start_position + step*step_size)
        servo_position(servo,position)
        time.sleep(time_delay)
    servo_position(servo,target_position)
    time.sleep(2)


#Upward rotation
angle_up = 30
target_position_servo_1_up = rotate_servo(angle_up)
target_position_servo_2_up = rotate_servo(-angle_up)
target_position_servo_3_up = rotate_servo(angle_up)
#Select rotational angle
def moveup(servo1,servo2,servo3):
    smooth_move(servo1,Initial_position,target_position_servo_1_up )
    smooth_move(servo2,Initial_position,target_position_servo_2_up)
    smooth_move(servo3,Initial_position,target_position_servo_3_up)

#up position to initial position
def moveup_to_initial(servo1,servo2,servo3):
    smooth_move(servo3,target_position_servo_3_up,Initial_position)
    smooth_move(servo2,target_position_servo_2_up,Initial_position)
    smooth_move(servo1,target_position_servo_1_up,Initial_position)
    
    

#Upward rotation
angle_down = -30
target_position_servo_1_down = rotate_servo(angle_down)
target_position_servo_2_down = rotate_servo(-angle_down)
target_position_servo_3_down = rotate_servo(angle_down)
#downwardward rotation
def movedown(servo1,servo2,servo3):
    #smooth_move(servo1,Initial_position,target_position_servo_1_down )
    smooth_move(servo2,Initial_position,target_position_servo_2_down)
    #smooth_move(servo3,Initial_position,target_position_servo_3_down)

#Move downwards to initial  position
def movedown_to_initial(servo1,servo2,servo3):
    #smooth_move(servo1,Initial_position,target_position_servo_1_down )
    smooth_move(servo2,target_position_servo_2_down,Initial_position)
    #smooth_move(servo3,Initial_position,target_position_servo_3_down)

#Set tsrget position
angle1=90
angle2=-90
angle3=0
servo_0_initial_position = rotate_servo(angle3)
servo_0_right_rotation = rotate_servo(angle1)
servo_0_left_rotation = rotate_servo(angle2)

####################################################################

# Reading CSV files

In [32]:
import os
import time
import pandas as pd
from datetime import datetime

target_array = np.array([5,6,7,8])

def is_file_fully_written(file_path, check_interval=0.1, max_checks=5):
    """Check if the file is still being written by monitoring its size."""
    previous_size = -1
    for _ in range(max_checks):
        current_size = os.path.getsize(file_path)
        if current_size == previous_size:
            return True
        previous_size = current_size
        time.sleep(check_interval)
    return False

def monitor_directory(directory):
    # Position to the initial
    initial_position(initial_position)
    # Get the initial list of files in the directory
    previous_files = set(os.listdir(directory))
    print("Monitoring directory:", directory)

    while True:
        # Get the current list of files in the directory
        current_files = set(os.listdir(directory))

        # Determine the new files by comparing current files with previous files
        new_files = current_files - previous_files
        
        if new_files:
            for new_file in new_files:
                print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - New file detected: {new_file}")
                file_path = os.path.join(directory, new_file)

                if is_file_fully_written(file_path):
                    try:
                        df = pd.read_csv(file_path, header=None)
                        #print(f"Dimensions of the data in {new_file}: {df.shape}")
                        time_array = df.to_numpy()
                        #print(f"Time Array shape {time_array.shape}")
                        feature_prepare = features(time_array,250,target_array,16).flatten()
                        feature_array = feature_prepare.reshape(-1,1).T
                        #print(f"feature_array.shape {feature_array.shape}")

                        # Cascaded Machine Learning Algorithm

                        result_5Hz = model1.predict_proba(feature_array).flatten()
                        if result_5Hz[0] > 0.9:
                            # Robot Arm Moves Up
                            initial_position(initial_position)
                            print("Moves Up")
                            moveup(1,2,3)
                        else:
                            result_6Hz = model2.predict_proba(feature_array).flatten()
                            if result_6Hz[0]>0.93:
                                initial_position(initial_position)
                                # Robot Arm Moves Down
                                movedown(1,2,3)
                                print("Moves Down")
                            else:
                                result_7Hz = model3.predict_proba(feature_array).flatten()
                                if result_7Hz[0]>0.81:
                                    initial_position(initial_position)
                                    # Robot Arm Moves Left
                                    smooth_move(0,1500,servo_0_left_rotation)
                                    print("Moves Left")
                                else:
                                    result_8Hz = model4.predict_proba(feature_array).flatten()
                                    if result_8Hz[0]>0.81:
                                        initial_position
                                        # Robot Arm Moves Right
                                        smooth_move(0,1500,servo_0_right_rotation)
                                        print("Move Right")
                                    else:
                                        # Rest Position
                                        print("Rest")
                        # Uncomment and customize the following code if you want to process the data further:
               
                    except pd.errors.EmptyDataError:
                        print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - New file detected but it is empty or contains no data: {new_file}")
                    except Exception as e:
                        print(f"Error reading {new_file}: {e}")
                else:
                    print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - File not fully written yet: {new_file}")

        # Update the previous file list
        previous_files = current_files

        # Wait for a short period before checking again
        time.sleep(0.01)

# Example usage
directory_to_monitor = r"F:\Engineering\Semester 8\EE406 - Undergraduate Project 2\csv files"
monitor_directory(directory_to_monitor)

Monitoring directory: F:\Engineering\Semester 8\EE406 - Undergraduate Project 2\csv files
2024-08-09 11:57:53 - New file detected: BioPotentialSignals_002.csv
Moves Up
2024-08-09 11:58:08 - New file detected: BioPotentialSignals_004.csv
Moves Up
2024-08-09 11:58:24 - New file detected: BioPotentialSignals_003.csv
Moves Up
2024-08-09 11:58:39 - New file detected: BioPotentialSignals_005.csv
Moves Up
2024-08-09 11:58:54 - New file detected: BioPotentialSignals_006.csv
Moves Left
2024-08-09 11:58:59 - New file detected: BioPotentialSignals_007.csv
Moves Up
2024-08-09 11:59:14 - New file detected: BioPotentialSignals_021.csv
Moves Up
2024-08-09 11:59:30 - New file detected: BioPotentialSignals_009.csv
Moves Up
2024-08-09 11:59:45 - New file detected: BioPotentialSignals_023.csv
Moves Left
2024-08-09 11:59:50 - New file detected: BioPotentialSignals_008.csv
Moves Up
2024-08-09 12:00:05 - New file detected: BioPotentialSignals_010.csv
Moves Up
2024-08-09 12:00:21 - New file detected: BioPote

KeyboardInterrupt: 

In [9]:
from scipy.io import loadmat
# load bigdata from local location
big_path = r"F:\Engineering\Semester 7\EE - 405 Undergraduate Project\26.01\bigdata.mat"
data = loadmat(big_path)
data.keys()
feature_array = data['feature_array']
unscaled_array = data['unscaled_array']

In [25]:
def make_predictor(feature_set):
    initial_position(initial_position)
    result_5Hz = model1.predict_proba(feature_set).flatten()
    if result_5Hz[0] > 0.9:
        initial_position(initial_position)
        # Robot Arm Moves Up
        moveup(1,2,3)
        print("Moves Up")   

    else:
        result_6Hz = model2.predict_proba(feature_array).flatten()
        if result_6Hz[0]>0.5:
            initial_position(initial_position)
            # Robot Arm Moves Down
            movedown(1,2,3)
            print("Moves Down")
        else:
            result_7Hz = model3.predict_proba(feature_array).flatten()
            if result_7Hz[0]>0.81:
                initial_position(initial_position)
                # Robot Arm Moves Left
                smooth_move(0,1500,servo_0_left_rotation)
                print("Moves Left")
            else:
                result_8Hz = model4.predict_proba(feature_array).flatten()
                if result_8Hz[0]>0.81:
                    initial_position(initial_position)
                    # Robot Arm Moves Right
                    smooth_move(0,1500,servo_0_right_rotation)
                    print("Move Right")
                else:
                    # Rest Position
                    print("Rest")

In [26]:
feature_array.shape

(16020, 192)

In [27]:
feature_set = feature_array[8000,:].reshape(-1,1).T
feature_set.shape
make_predictor(feature_set)

Moves Down
